<a href="https://colab.research.google.com/github/ImagingDataCommons/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/dicomSEGMaps/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook generates the config map used for creating DICOM SEG files for segmentations created by TotalSegmentator
- The snomed codes for label and label maps are downloaded from official TotalSegmentator repo
- Colors are then chosen from MhubAI
- Lastly some colors are replaced by the ones chosen by Deepa

###**Importing Packages**

In [ ]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import numpy as np
import ast
from natsort import natsorted

###**Runtime Environment**

In [ ]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

**map_to_binary.py contains the mapping of labels and body parts and totalsegmentator_snomed_mapping.csv contains the body part to snomed codes.
Parsing only total_v1 label map as we have colors only for the v1**

In [ ]:
try:
  os.remove(f'{curr_dir}/map_to_binary.py')
except OSError:
  pass
!wget -q  https://raw.githubusercontent.com/wasserth/TotalSegmentator/v2.0.5/totalsegmentator/map_to_binary.py
!wget -q  https://raw.githubusercontent.com/wasserth/TotalSegmentator/1691bb8cd27a9ab78c2da3acef4dddf677c7dd24/resources/totalsegmentator_snomed_mapping.csv
import map_to_binary

In [ ]:
label_id_body_part_data = map_to_binary.class_map['total_v1']
label_id_body_part_data_df = pd.DataFrame(list(label_id_body_part_data.items()), columns=['labelID', 'Structure'])
#label_id_body_part_data_df= label_id_body_part_data_df.astype(str)
label_id_body_part_data_df

In [ ]:
totalsegmentator_snomed_mapping_df=pd.read_csv('totalsegmentator_snomed_mapping.csv', encoding='utf-8',dtype='str')
totalsegmentator_snomed_mapping_df

###Merge label map df and codes df using Structure name as key

In [ ]:
totalseg_snomed_mapping_with_labels_df=pd.merge(totalsegmentator_snomed_mapping_df,label_id_body_part_data_df, how='left', left_on='Structure', right_on='Structure' )
totalseg_snomed_mapping_with_labels_df

These colors were originally chosen from mhubai' repo. However they labeled all parts as '1' so we can't use them directly

`https://raw.githubusercontent.com/vkt1414/models/main/models/totalsegmentator/config/dicomseg_metadata_whole.json`

Instead, we will use
`https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/main/configs/TotalSegmentator/dicomseg_metadata_whole_slicerAsRef.json`



In [ ]:
!wget -q https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/main/configs/TotalSegmentator/dicomseg_metadata_whole_slicerAsRef.json

In [ ]:
with open('dicomseg_metadata_whole_slicerAsRef.json', 'r') as file:
    json_data = json.load(file)

# Initialize an empty list to store each DataFrame
data_frames = []

# Loop through each item in 'segmentAttributes'
for item in json_data['segmentAttributes']:
    # Normalize the item and append the DataFrame to the list
    item_data = pd.json_normalize(item).reset_index(drop=True)
    data_frames.append(item_data)

# Concatenate all DataFrames in the list
mhubai_colors = pd.concat(data_frames, ignore_index=True)

mhubai_colors=mhubai_colors[['labelID', 'recommendedDisplayRGBValue']]
#mhubai_colors=mhubai_colors.astype(str)
mhubai_colors

###Colors chosen by Deepa, for differentiating vertebrae and a few other body parts

In [ ]:
!wget -q https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/main/configs/TotalSegmentator/distinct_vertebrae_ribs_rgb_colors_TotalSegmentator.csv

In [ ]:
distinct_vertebrae_ribs_colors=pd.read_csv('distinct_vertebrae_ribs_rgb_colors_TotalSegmentator.csv')
distinct_vertebrae_ribs_colors

In [ ]:
distinct_vertebrae_ribs_colors.dtypes

In [ ]:
distinct_vertebrae_ribs_colors_with_labels=pd.merge(distinct_vertebrae_ribs_colors,totalseg_snomed_mapping_with_labels_df,how='left',left_on='totalseg_labelname', right_on='Structure')
distinct_vertebrae_ribs_colors_with_labels=distinct_vertebrae_ribs_colors_with_labels[['labelID','recommendedDisplayRGBValue']]
#distinct_vertebrae_ribs_colors_with_labels=distinct_vertebrae_ribs_colors_with_labels.astype(str)
distinct_vertebrae_ribs_colors_with_labels

##Replace mhubai colors with Deepa's colors whereever possible

In [ ]:
# Specify suffixes
mhubai_colors_with_distinct_colors = pd.merge(mhubai_colors, distinct_vertebrae_ribs_colors_with_labels, how='left', left_on='labelID', right_on='labelID', suffixes=('_mhub', '_distinct'))

# Fill NaN values in the 'distinct' column with 'mhub' values
mhubai_colors_with_distinct_colors['recommendedDisplayRGBValue_distinct'].fillna(mhubai_colors_with_distinct_colors['recommendedDisplayRGBValue_mhub'], inplace=True)

mhubai_colors_with_distinct_colors.drop(columns=['recommendedDisplayRGBValue_mhub'], inplace=True)

face_rgb_value = [255, 182, 193]

# Create a mask for the rows where labelID is 93
mask = mhubai_colors_with_distinct_colors['labelID'] == 93

# Create an iterable with the same length as the number of rows you're updating
face_rgb_value_iterable = pd.Series([face_rgb_value] * sum(mask))

# Set the RGB value for labelID 93
mhubai_colors_with_distinct_colors.loc[mask, 'recommendedDisplayRGBValue_distinct'] = face_rgb_value_iterable.values
mhubai_colors_with_distinct_colors

In [ ]:
# Function to convert string to list
def convert_str_to_list(s):
    if isinstance(s, list):
        # If s is already a list, return it as it is
        return s
    try:
        # Try to convert string representation of list to actual list
        return ast.literal_eval(s)
    except ValueError:
        # If ValueError occurs, split the string on ',' and convert each item to int
        return [int(i) for i in s.split(',')]

# Apply the function to the 'recommendedDisplayRGBValue_distinct' column
mhubai_colors_with_distinct_colors['recommendedDisplayRGBValue_distinct'] = mhubai_colors_with_distinct_colors['recommendedDisplayRGBValue_distinct'].apply(convert_str_to_list)
mhubai_colors_with_distinct_colors

In [ ]:
final_df=pd.merge(totalseg_snomed_mapping_with_labels_df,mhubai_colors_with_distinct_colors,how='left',on='labelID')
final_df=final_df.rename(columns={'recommendedDisplayRGBValue_distinct':'recommendedDisplayRGBValue'})
final_df_without_labelid=final_df.drop(columns=["labelID"])

final_df

In [ ]:
final_df_without_labelid.to_csv('totalsegmentator_snomed_mapping_with_partial_colors.csv')

In [ ]:
dcmqi_df=final_df[final_df['recommendedDisplayRGBValue'].notnull()]
dcmqi_df = dcmqi_df[dcmqi_df.columns.drop(list(dcmqi_df.filter(regex='Anatomic')))]
dcmqi_df = dcmqi_df.drop(columns=['Structure'])
dcmqi_df['labelID']=dcmqi_df['labelID'].astype(int)
#dcmqi_df['recommendedDisplayRGBValue']=dcmqi_df['recommendedDisplayRGBValue'].astype(ast.literal_eval)
dcmqi_df['SegmentDescription'] = dcmqi_df.apply(
    lambda row: row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] + ' ' + row['SegmentedPropertyTypeCodeSequence.CodeMeaning']
    if pd.notnull(row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']) else row['SegmentedPropertyTypeCodeSequence.CodeMeaning'], axis=1)
dcmqi_df['SegmentAlgorithmName']='TotalSegmentator v1.5.6'
dcmqi_df['SegmentAlgorithmType']='AUTOMATIC'
# dcmqi_df['SegmentedPropertyCategoryCodeSequence.CodeValue']=dcmqi_df['SegmentedPropertyCategoryCodeSequence.CodeValue'].astype(str)
# dcmqi_df['SegmentedPropertyTypeModifierCodeSequence.CodeValue']=dcmqi_df['SegmentedPropertyTypeModifierCodeSequence.CodeValue'].astype(str)
# dcmqi_df['SegmentedPropertyTypeCodeSequence.CodeValue'] = dcmqi_df['SegmentedPropertyTypeCodeSequence.CodeValue'].astype(str)
dcmqi_df.fillna('', inplace=True)
dcmqi_df

In [ ]:
dcmqi_df.to_csv('test.csv')

In [ ]:
merged_df_json = json.loads(dcmqi_df.to_json(orient='records'))

for segment in merged_df_json:
    segment['SegmentedPropertyCategoryCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeMeaning')
    }
    segment['SegmentedPropertyTypeCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyTypeCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyTypeCodeSequence.CodeMeaning')
    }
    if ('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator' in segment and
        (segment['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeValue'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'].strip())):
        segment['SegmentedPropertyTypeModifierCodeSequence'] = {
            'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'),
            'CodeValue': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeValue'),
            'CodeMeaning': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeMeaning')
        }
    else:
        for key in ['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']:
            if key in segment:
                del segment[key]
final_json = {
    "BodyPartExamined": "CHEST",
    "ClinicalTrialCoordinatingCenterName": "dcmqi",
    "ClinicalTrialSeriesID": "0",
    "ClinicalTrialTimePointID": "1",
    "ContentCreatorName": "IDC",
    "ContentDescription": "Image segmentation",
    "ContentLabel": "SEGMENTATION",
    "InstanceNumber": "1",
    "SeriesDescription": "TotalSegmentator Segmentation",
    "SeriesNumber": "42",
    "segmentAttributes": [merged_df_json]
}
with open('dicomseg_metadata_totalseg_v1.json', 'w') as file:
    json.dump(final_json, file, indent=4)

In [ ]:
import yaml
with open('dicomseg_metadata_totalseg_v1.yaml', 'w') as file:
    yaml.dump(final_json, file)
